In [ ]:
# Dependencies
import pandas as pd

In [ ]:
# Constants


In [ ]:
# Aaron's Code

In [ ]:
## Import Billboard Hits
filepath = "../data/external/billboard_hits/"
def import_billboard_hit(filename):
    columns = ["Date"]
    columns.extend(range(1,101))
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1", names=columns)

def import_billboard_feature(filename):
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1")

billboard_hits = []
billboard_features = []
for year in range(1960, 2020, 10):
    print(f"Importing {year}'s top 100.")
    billboard_hit = import_billboard_hit(f"billboard{year}")
    print(f"Importing {year}'s features.")
    billboard_feature = import_billboard_feature(f"billboard{year}FeaturesDatabase")
    
    billboard_hits.append(billboard_hit)
    billboard_features.append(billboard_feature)

In [ ]:
billboard_hits[0].head()

In [ ]:
billboard_features[0].head()

In [ ]:
# Combine Hits Dataframes
all_billboard_hits = pd.concat(billboard_hits)
all_billboard_hits = all_billboard_hits.reset_index()
all_billboard_hits["Date"].nunique()

In [ ]:
# Combine Features Dataframes
all_billboard_features = pd.concat(billboard_features)
all_billboard_features["Track"].nunique()

In [ ]:
# Split the date
split_dates = all_billboard_hits["Date"].str.split(pat = "-", expand=True)
split_dates.columns = ["Year", "Month", "Day"]
all_billboard_hits["Year"] = split_dates["Year"]
all_billboard_hits["Month"] = split_dates["Month"]
all_billboard_hits["Day"] = split_dates["Day"]
all_billboard_hits.head()

In [ ]:
# Reform the hits table to split up song/artist to be able to merge
def split_song_artist(song_artist):
    song_artist = song_artist.replace('"', "'")
    split = song_artist.split("', '")
    song = split[0].replace("['", "")
    artist = split[1].replace("']", "")
    return (song, artist)

In [ ]:
song_datas = []
for index, row in all_billboard_hits.iterrows():
    for hit_num in range(1, 101):
        song_artist = row[hit_num]
        try:
            (song, artist) = split_song_artist(song_artist)
        except:
            print(f"Couldn't parse {song_artist}")
            song = ""
            artist = ""
        song_data = {
            "Track" : song,
            "Artist" : artist,
            "Placement" : hit_num,
            "Year" : row["Year"],
            "Month" : row["Month"],
            "Day" : row["Day"],
        }
        song_datas.append(song_data)

In [ ]:
all_hits = pd.DataFrame(song_datas)
all_hits.head()

In [ ]:
# Merge Billboard Data and Spotify Data
final_music = pd.merge(all_hits, all_billboard_features, on=["Track", "Artist"])
columns_to_drop = ["URI", "mode", "duration_ms", "instrumentalness", 
                   "time_signature", "chorusHit", "sections"]
final_music = final_music.drop(columns = columns_to_drop)
final_music = final_music.dropna()
final_music.head()

In [ ]:
# Save to CSV
final_music.to_csv("../data/billboard_hits.csv")

In [ ]:
# End of Aaron's Code

In [ ]:
# Albin's Code

In [ ]:
# End of Albin's Code

In [ ]:
# Kelsey's Code

In [ ]:
# End of Kelsey's Code

In [ ]:
# Will's Code

In [ ]:
# End of Will's Code